# Workflow 8 - module 2 - Diabetes

This notebook implements Module 2 of Workflow 8 for diabetes genes and tissues. See <a href="https://docs.google.com/presentation/d/1IkAzjSrOMzOLx5z8GqRVKmVd1GKrpIEb_xF4g4RlI1U/edit?usp=sharing">here</a> for an overview of Workflow 8. Module 2 takes **genes and tissues** as input and **interacting genes** and/or a **gene interaction/similarrity matrix** that can be used as input for module 3 (DDOT). Module 2 uses Google BigQuery. Boilerplate code (API calls for instance) is wrapped in `wf8_module2.py`, which contains API calls written by John Earls and Theo Knijnenburg.  

Notebook written by: Samson Fong, John Earls, Theo Knijnenburg, Chris Churas and Aaron Gary. 

## Libraries and such

In [1]:
%load_ext autoreload 
%autoreload 2

import json
from pprint import pprint
from wf8_module1 import doid_to_genes_and_tissues
from wf8_module2 import call_biggim
import numpy as np
import pandas as pd
import time

The main function **call_biggim** has seven input arguments of which only the first two are required
1. genes [required]  - List of input genes as a set of NCBI genes, e.g. genes = ['2188', '79728', '7124']
2. tissues [required] - LIst of tissues, e.g. tissues = ['pancreas', 'liver']
3. limit=1000000 - Maximum number of rows returned. Make smaller for quicker testing and smaller tables
4. average_columns=False - If tissues are associated with multiple columns average the scores into one columns called 'mean'
5. query_id2=False - List of input genes. Only pairs that are constituted of genes in 'genes' and 'query_id2' are returned
6. return_genes=False - Return new genes (interacting with the original set of genes) as a list
7. N=250 - Number of new genes returned

# Testing for Diabetes

Tissue and genes are from MarcinvG/Sui.

## Getting Interacting Genes

 We are asking for N=100 genes that interact with the 10 seed genes.

In [2]:
genes = ['10239','1029','145741','169026','3087','4544','54901','6927','6934','8872']
tissues =  ['pancreas', 'liver', 'muscle', 'adipose']
genes2 = call_biggim(genes, tissues, average_columns=True, return_genes=True, N=100)


Sent: GET http://biggim.ncats.io/api/metadata/tissue/pancreas?None
Sent: GET http://biggim.ncats.io/api/metadata/tissue/liver?None
Sent: GET http://biggim.ncats.io/api/metadata/tissue/muscle?None
Sent: GET http://biggim.ncats.io/api/metadata/tissue/adipose?None
Returned 21 Big GIM columns
['GIANT_pancreas_KnownFunctionalInteraction', 'GIANT_hepatocyte_ProbabilityOfFunctionalInteraction', 'GIANT_hepatocyte_KnownFunctionalInteraction', 'GIANT_liver_KnownFunctionalInteraction', 'GIANT_skeletal_muscle_KnownFunctionalInteraction', 'GIANT_skeletal_muscle_ProbabilityOfFunctionalInteraction', 'GIANT_pancreatic_islet_KnownFunctionalInteraction', 'GTEx_Pancreas_Correlation', 'GIANT_pancreas_ProbabilityOfFunctionalInteraction', 'GTEx_Muscle_Correlation', 'GIANT_liver_ProbabilityOfFunctionalInteraction', 'GTEx_Liver_Correlation', 'GIANT_cardiac_muscle_ProbabilityOfFunctionalInteraction', 'GIANT_myometrium_KnownFunctionalInteraction', 'GIANT_pancreatic_islet_ProbabilityOfFunctionalInteraction', 'GI

In [3]:
print('All genes')
print(genes2)

All genes
['10190', '1021', '10239', '1029', '1029', '10395', '10436', '10492', '10528', '10539', '10574', '10576', '10606', '10694', '10799', '10923', '10971', '10987', '11030', '11137', '11164', '11222', '1434', '145741', '158', '169026', '1736', '2079', '2107', '2247', '2261', '2271', '22919', '22948', '23560', '24140', '2487', '2534', '26135', '2618', '2665', '27258', '27303', '2737', '28985', '29889', '3087', '3087', '334', '3396', '3608', '367', '3735', '3838', '4172', '4544', '471', '4853', '4921', '5052', '5223', '5464', '54901', '55759', '5631', '5663', '5682', '5683', '5684', '5685', '5686', '5689', '5695', '5701', '5705', '5707', '5718', '5901', '5902', '6059', '6418', '6632', '6637', '6731', '6742', '6790', '6927', '6934', '6934', '6950', '7019', '7334', '7341', '7371', '7411', '7520', '7818', '79084', '8520', '8533', '8624', '8660', '87178', '8833', '8872', '8872', '8886', '908', '9184', '9521', '9775', '9790']


In [4]:
print('Here are the new genes')
print(list(set(genes2)-set(genes)))

Here are the new genes
['7371', '11164', '5689', '5701', '7019', '8833', '22948', '10574', '23560', '2079', '10436', '5683', '3735', '6950', '8624', '1434', '8886', '10190', '4172', '4853', '7520', '28985', '158', '5718', '7334', '6418', '8520', '5684', '79084', '8533', '27258', '7341', '2247', '5685', '10694', '5663', '6731', '6059', '10987', '26135', '471', '6742', '5707', '7818', '27303', '2271', '3396', '5052', '11030', '10923', '5464', '22919', '2261', '11137', '55759', '9184', '5902', '9775', '1736', '6632', '10971', '3608', '2534', '908', '2107', '24140', '9521', '10528', '5686', '5695', '2487', '10395', '10539', '5682', '2618', '29889', '2665', '6637', '10799', '3838', '1021', '5631', '367', '5901', '10606', '5705', '10576', '7411', '6790', '8660', '9790', '2737', '10492', '334', '4921', '5223', '87178', '11222']


## Running Big GIM for the second round, now producing the interaction matrix (df) that can serve as an input for DDOT

In [5]:
df = call_biggim(genes2, tissues, average_columns=True, query_id2=genes2)

Sent: GET http://biggim.ncats.io/api/metadata/tissue/pancreas?None
Sent: GET http://biggim.ncats.io/api/metadata/tissue/liver?None
Sent: GET http://biggim.ncats.io/api/metadata/tissue/muscle?None
Sent: GET http://biggim.ncats.io/api/metadata/tissue/adipose?None
Returned 21 Big GIM columns
['GIANT_pancreas_KnownFunctionalInteraction', 'GIANT_hepatocyte_ProbabilityOfFunctionalInteraction', 'GIANT_hepatocyte_KnownFunctionalInteraction', 'GIANT_liver_KnownFunctionalInteraction', 'GIANT_skeletal_muscle_KnownFunctionalInteraction', 'GIANT_skeletal_muscle_ProbabilityOfFunctionalInteraction', 'GIANT_pancreatic_islet_KnownFunctionalInteraction', 'GTEx_Pancreas_Correlation', 'GIANT_pancreas_ProbabilityOfFunctionalInteraction', 'GTEx_Muscle_Correlation', 'GIANT_liver_ProbabilityOfFunctionalInteraction', 'GTEx_Liver_Correlation', 'GIANT_cardiac_muscle_ProbabilityOfFunctionalInteraction', 'GIANT_myometrium_KnownFunctionalInteraction', 'GIANT_pancreatic_islet_ProbabilityOfFunctionalInteraction', 'GI

In [11]:
print(df)

     Gene1  Gene2      mean
0     2177   2176  0.126488
1    11065    675  0.214483
2    29089   2188  0.089574
3     5889   3161  0.143898
4    79728   7516  0.057075
5    11065   2178  0.095352
6     5889   2176  0.049106
7     5889   2178  0.085579
8    55120    580  0.216469
9    55215   2175  0.186829
10    2175   1479  0.079013
11    2176    675  0.078430
12   81620   4939  0.066903
13   83990  79728  0.071995
14    4939    675  0.071992
15    7516   3161  0.081837
16   84464  57697  0.080099
17   29089    580  0.187491
18    2178    580  0.067098
19   57697  55215  0.112632
20    7124   4599  0.073832
21   84464   2178  0.058176
22   57697   7516  0.084148
23    5888   2176  0.069216
24    2176   2175  0.256351
25    2188    675  0.071333
26   55215   7516  0.094710
27   84464   2175  0.095109
28   79728  57697  0.081342
29   29089   3161  0.265825
..     ...    ...       ...
299  83990    580  0.092860
300   8317   2189  0.230804
301   9246   7124  0.096379
302   2189   2176  0